In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import cv2
import copy
import os
import numpy as np

from scipy import interpolate

def imgshow(img):
    if len(img.shape) == 3:
        plt.imshow(img[:, :, ::-1])
    else:
        plt.imshow(img,cmap='gray',vmin=0, vmax=255)
    plt.show() 


In [ ]:
# with open('unfishmap2.csv', 'wb') as f:
#     np.savetxt(f, unfishmap2, fmt='%d', delimiter=",",newline='\n')

raw = np.genfromtxt('fisheye_angle calibration_raw.csv',delimiter=',',skip_header=1)

In [ ]:
x,y = raw[::,0],raw[::,1]

fig, ax = plt.subplots()
ax.scatter(x, y)
plt.show()

x,y = raw[::,2],raw[::,3]

fig, ax = plt.subplots()
ax.scatter(x, y)
plt.xlim([0, 639])
plt.ylim([0, 479])
plt.show()


In [ ]:
'''
bad:
-angle offset
-pixel noise
-missing values

good:
-many samples
-angles on grid
-missing values mostly in 1 dimension

-tasks:
-fix offset
-convert to look-up table, quartered
-reverse table
-smooth data
-fill missing datapoints
-extrapolate?
'''

In [ ]:
line = raw[raw[::,0]==-55]
dep,indep = line[::,2],line[::,1]

fig, ax = plt.subplots()
ax.scatter(x, y)
plt.xlim([0, 639])
plt.ylim([0, 479])
plt.show()

fit_func = np.poly1d(np.polyfit(indep,dep,2))
dep_fit = fit_func(indep)

indep_expanded = np.arange(-55,25+1,5)
dep_fit_expanded = fit_func(indep_expanded)

fig, ax = plt.subplots()
ax.scatter(indep, dep)
plt.show()

fig, ax = plt.subplots()
ax.scatter(indep, dep_fit)
plt.show()


fig, ax = plt.subplots()
ax.scatter(indep_expanded, dep_fit_expanded)
plt.show()



In [ ]:
#smoothing:
raw_copy = raw.copy()

for i in range(-55,55+1,5):    
    line = raw_copy[raw_copy[::,0]==i]
    dep,indep = line[::,2],line[::,1]

    fit_func = np.poly1d(np.polyfit(indep,dep,2))
    dep_fit = fit_func(indep)
    
    raw_copy[raw_copy[::,0]==i,2] = dep_fit

#     indep_expanded = np.arange(-55,25+1,5)
#     dep_fit_expanded = fit_func(indep_expanded)

x,y = raw_copy[::,2],raw_copy[::,3]
fig, ax = plt.subplots()
ax.scatter(x, y)
plt.xlim([0, 639])
plt.ylim([0, 479])
plt.show()

for i in range(-55,25+1,5):    
    line = raw_copy[raw_copy[::,1]==i]
    dep,indep = line[::,3],line[::,0]

    fit_func = np.poly1d(np.polyfit(indep,dep,2))
    dep_fit = fit_func(indep)
    
    raw_copy[raw_copy[::,1]==i,3] = dep_fit

#     indep_expanded = np.arange(-55,25+1,5)
#     dep_fit_expanded = fit_func(indep_expanded)

x,y = raw_copy[::,2],raw_copy[::,3]
fig, ax = plt.subplots()
ax.scatter(x, y)
plt.xlim([0, 639])
plt.ylim([0, 479])
plt.show()




In [ ]:
x,y = raw_copy[::,2],raw_copy[::,3]
fig, ax = plt.subplots()
ax.scatter(x, y)
ax.scatter(319.5,239.5)
plt.xlim([0, 639])
plt.ylim([0, 479])
plt.show()

# x,y = raw_copy[::,2],raw_copy[::,3]
# z = raw_copy[::,0]
# f = interpolate.interp2d(x, y, z, kind='linear')
# x_angle_offset = f(319.5, 239.5)
# print(x_angle_offset)
# # z = raw_copy[::,1]
# # f = interpolate.interp2d(x, y, z, kind='linear')
# # y_angle_offset = f(319.5, 239.5)

# # print(x_angle_offset,y_angle_offset)

#check if point exists
#(raw_copy[::,2]==319.5)  and (raw_copy[::,3]==239.5)

# find surrounding points



In [ ]:
x,y = raw_copy[::,2],raw_copy[::,3]
distances_sqrd = (x-319.5)**2 + (y-239.5)**2

x,y = raw_copy[::,2],raw_copy[::,3]
fig, ax = plt.subplots()
ax.scatter(x, y,distances_sqrd/10000)
ax.scatter(319.5,239.5)
plt.xlim([0, 639])
plt.ylim([0, 479])
plt.show()

np.min(distances_sqrd)

q1i = np.logical_and(raw_copy[::,2] < 319.5,raw_copy[::,3] < 239.5)
q1 =  raw_copy[q1i]
x,y = q1[::,2],q1[::,3]
q1_near_i = np.argmin(distances_sqrd[q1i])
q1_near = q1[q1_near_i]
print(q1_near)

fig, ax = plt.subplots()
ax.scatter(x, y,distances_sqrd[q1i]/10000)
ax.scatter(319.5,239.5)
ax.scatter(q1_near[2],q1_near[3])

plt.xlim([0, 639])
plt.ylim([0, 479])
plt.show()

q2i = np.logical_and(raw_copy[::,2] > 319.5,raw_copy[::,3] < 239.5)
q2 =  raw_copy[q2i]
q2_near = q2[np.argmin(distances_sqrd[q2i])]
q3i = np.logical_and(raw_copy[::,2] < 319.5,raw_copy[::,3] > 239.5)
q3 =  raw_copy[q3i]
q3_near = q3[np.argmin(distances_sqrd[q3i])]
q4i = np.logical_and(raw_copy[::,2] > 319.5,raw_copy[::,3] > 239.5)
q4 =  raw_copy[q4i]
q4_near = q4[np.argmin(distances_sqrd[q4i])]


x,y = raw_copy[::,2],raw_copy[::,3]

fig, ax = plt.subplots()
ax.scatter(x, y,distances_sqrd/10000)
ax.scatter(319.5,239.5)
ax.scatter(q1_near[2],q1_near[3])
ax.scatter(q2_near[2],q2_near[3])
ax.scatter(q3_near[2],q3_near[3])
ax.scatter(q4_near[2],q4_near[3])

plt.xlim([0, 639])
plt.ylim([0, 479])
plt.show()

